# 02. Learning LangGraph - Chat Executor

In [1]:
# !pip install --quiet -U langchain langchain_openai langgraph langchainhub langchain_experimental

modified from https://github.com/langchain-ai/langgraph/blob/main/examples/chat_agent_executor_with_function_calling/base.ipynb

In [1]:
# importing os module for environment variables
import os
# importing necessary functions from dotenv library
from dotenv import load_dotenv, dotenv_values 
# loading variables from .env file
load_dotenv()

True

In [2]:
api_key = os.getenv('PRACTICE_KEY')
langchain_api_key = os.getenv('LANGCHAIN_KEY')

In [3]:
import os
# from google.colab import userdata

os.environ["OPENAI_API_KEY"] = api_key

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = langchain_api_key
os.environ["LANGCHAIN_PROJECT"] = "LangGraph_02"

## The model

In [6]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(temperature=0, streaming=True)

## Tools

In [7]:
from langchain.tools import BaseTool, StructuredTool, Tool, tool
import random

@tool("lower_case", return_direct=True)
def to_lower_case(input:str) -> str:
  """Returns the input as all lower case."""
  return input.lower()

@tool("random_number", return_direct=True)
def random_number_maker(input:str) -> str:
    """Returns a random number between 0-100. input the word 'random'"""
    return random.randint(0, 100)

tools = [to_lower_case,random_number_maker]

In [8]:
from langgraph.prebuilt.tool_executor import ToolExecutor

tool_executor = ToolExecutor(tools)

In [9]:
from langchain.tools.render import format_tool_to_openai_function

functions = [format_tool_to_openai_function(t) for t in tools]
model = model.bind_functions(functions)

c:\Users\bhaiyasingh\Documents\llm_env\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The function `format_tool_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 0.3.0. Use langchain_core.utils.function_calling.convert_to_openai_function() instead.
  warn_deprecated(


## AgentState

In [10]:
from typing import TypedDict, Annotated, Sequence
import operator
from langchain_core.messages import BaseMessage


class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]

## Nodes

In [11]:
from langchain_core.agents import AgentFinish
from langgraph.prebuilt import ToolInvocation
import json
from langchain_core.messages import FunctionMessage

# Define the function that determines whether to continue or not
def should_continue(state):
    messages = state['messages']
    last_message = messages[-1]
    # If there is no function call, then we finish
    if "function_call" not in last_message.additional_kwargs:
        return "end"
    # Otherwise if there is, we continue
    else:
        return "continue"

# Define the function that calls the model
def call_model(state):
    messages = state['messages']
    response = model.invoke(messages)
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}

# Define the function to execute tools
def call_tool(state):
    messages = state['messages']
    # Based on the continue condition
    # we know the last message involves a function call
    last_message = messages[-1]

    print('last_message = ', last_message)

    # We construct an ToolInvocation from the function_call
    action = ToolInvocation(
        tool=last_message.additional_kwargs["function_call"]["name"],
        tool_input=json.loads(last_message.additional_kwargs["function_call"]["arguments"]),
    )
    print(f"The agent action is {action}")
    # We call the tool_executor and get back a response
    response = tool_executor.invoke(action)
    print(f"The tool result is: {response}")
    # We use the response to create a FunctionMessage
    function_message = FunctionMessage(content=str(response), name=action.tool)
    # We return a list, because this will get added to the existing list
    return {"messages": [function_message]}

## Graph

In [12]:
from langgraph.graph import StateGraph, END
# Define a new graph
workflow = StateGraph(AgentState)

# Define the two nodes we will cycle between
workflow.add_node("agent", call_model)
workflow.add_node("action", call_tool)

# Set the entrypoint as `agent` where we start
workflow.set_entry_point("agent")

# We now add a conditional edge
workflow.add_conditional_edges(
    # First, we define the start node. We use `agent`.
    # This means these are the edges taken after the `agent` node is called.
    "agent",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
    # Finally we pass in a mapping.
    # The keys are strings, and the values are other nodes.
    # END is a special node marking that the graph should finish.
    # What will happen is we will call `should_continue`, and then the output of that
    # will be matched against the keys in this mapping.
    # Based on which one it matches, that node will then be called.
    {
        # If `tools`, then we call the tool node.
        "continue": "action",
        # Otherwise we finish.
        "end": END
    }
)

# We now add a normal edge from `tools` to `agent`.
# This means that after `tools` is called, `agent` node is called next.
workflow.add_edge('action', 'agent')

# Finally, we compile it!
# This compiles it into a LangChain Runnable,
# meaning you can use it as you would any other runnable
app = workflow.compile()

## Run it

In [13]:
from langchain_core.messages import HumanMessage, SystemMessage
# inputs = {"input": "give me a random number and then write in words and make it lower case", "chat_history": []}

system_message = SystemMessage(content="you are a helpful assistant")
user_01 = HumanMessage(content="give me a random number and then write in words and make it lower case")
# user_01 = HumanMessage(content="plear write 'Merlion' in lower case")
# user_01 = HumanMessage(content="what is a Merlion?")

inputs = {"messages": [system_message,user_01]}

app.invoke(inputs)

last_message =  content='' additional_kwargs={'function_call': {'arguments': '{"input":"random"}', 'name': 'random_number'}} response_metadata={'finish_reason': 'function_call'} id='run-8eefd2c8-c5f5-4f56-84ac-a820c0e0aa20-0'
The agent action is tool='random_number' tool_input={'input': 'random'}
The tool result is: 43
last_message =  content='' additional_kwargs={'function_call': {'arguments': '{"input":"forty-three"}', 'name': 'lower_case'}} response_metadata={'finish_reason': 'function_call'} id='run-417bfcd9-16d6-48f3-80ce-44bae85c67dd-0'
The agent action is tool='lower_case' tool_input={'input': 'forty-three'}
The tool result is: forty-three


{'messages': [SystemMessage(content='you are a helpful assistant'),
  HumanMessage(content='give me a random number and then write in words and make it lower case'),
  AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"input":"random"}', 'name': 'random_number'}}, response_metadata={'finish_reason': 'function_call'}, id='run-8eefd2c8-c5f5-4f56-84ac-a820c0e0aa20-0'),
  FunctionMessage(content='43', name='random_number'),
  AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"input":"forty-three"}', 'name': 'lower_case'}}, response_metadata={'finish_reason': 'function_call'}, id='run-417bfcd9-16d6-48f3-80ce-44bae85c67dd-0'),
  FunctionMessage(content='forty-three', name='lower_case'),
  AIMessage(content='The random number is 43, and in words, it is "forty-three" in lower case.', response_metadata={'finish_reason': 'stop'}, id='run-0fb914a9-71dc-4b74-afb5-892488fd7e80-0')]}

In [14]:
from langchain_core.messages import HumanMessage, SystemMessage
# inputs = {"input": "give me a random number and then write in words and make it lower case", "chat_history": []}

system_message = SystemMessage(content="you are a helpful assistant")
# user_01 = HumanMessage(content="give me a random number and then write in words and make it lower case")
user_01 = HumanMessage(content="plear write 'Merlion' in lower case")
# user_01 = HumanMessage(content="what is a Merlion?")

inputs = {"messages": [system_message,user_01]}

app.invoke(inputs)

last_message =  content='' additional_kwargs={'function_call': {'arguments': '{"input":"Merlion"}', 'name': 'lower_case'}} response_metadata={'finish_reason': 'function_call'} id='run-25351cc4-b911-4b1d-9c90-eb33ce9ff96f-0'
The agent action is tool='lower_case' tool_input={'input': 'Merlion'}
The tool result is: merlion


{'messages': [SystemMessage(content='you are a helpful assistant'),
  HumanMessage(content="plear write 'Merlion' in lower case"),
  AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"input":"Merlion"}', 'name': 'lower_case'}}, response_metadata={'finish_reason': 'function_call'}, id='run-25351cc4-b911-4b1d-9c90-eb33ce9ff96f-0'),
  FunctionMessage(content='merlion', name='lower_case'),
  AIMessage(content='The word "Merlion" in lower case is "merlion".', response_metadata={'finish_reason': 'stop'}, id='run-d0202527-375b-4c58-b6b1-58c53024e17b-0')]}

In [15]:
from langchain_core.messages import HumanMessage, SystemMessage
# inputs = {"input": "give me a random number and then write in words and make it lower case", "chat_history": []}

system_message = SystemMessage(content="you are a helpful assistant")
# user_01 = HumanMessage(content="give me a random number and then write in words and make it lower case")
# user_01 = HumanMessage(content="plear write 'Merlion' in lower case")
user_01 = HumanMessage(content="what is a Merlion?")

inputs = {"messages": [system_message,user_01]}

app.invoke(inputs)

{'messages': [SystemMessage(content='you are a helpful assistant'),
  HumanMessage(content='what is a Merlion?'),
  AIMessage(content='A Merlion is a mythical creature with the head of a lion and the body of a fish. It is a symbol of Singapore and is often seen as a mascot representing the city-state. The Merlion is a popular tourist attraction in Singapore, with statues of the creature located at various spots around the country.', response_metadata={'finish_reason': 'stop'}, id='run-3a50c452-90c6-470e-bebf-514823d495e8-0')]}